# 특정단어가 언급된 트위터 개수 크롤링하기 - 한 달씩 (ex. 연남동)

참고 : [왕형준님 블로그](https://medium.com/@whj2013123218/%ED%8C%8C%EC%9D%B4%EC%8D%AC%EC%9D%84-%EC%9D%B4%EC%9A%A9%ED%95%9C-twitter-%ED%81%AC%EB%A1%A4%EB%A7%81-576f7b098daf)

중복되는 코드를 없앴고 연도별 데이터를 보는 우리 프로젝트에 맞게 코드를 조금 수정했다

total_twitter_permonth 함수 : 기간, 동, 트윗 수, 하트 수, 연도로 데이터프레임을 반환

In [4]:
from bs4 import BeautifulSoup
import requests
from selenium import webdriver
from selenium.webdriver.firefox.firefox_binary import  FirefoxBinary
from selenium.webdriver.common.desired_capabilities import  DesiredCapabilities
import time
from selenium.webdriver.common.keys import Keys
import datetime as dt
import pandas as pd
import numpy as np
from dateutil.relativedelta import relativedelta
import re
import warnings
warnings.filterwarnings('ignore')

아래 코드를 실행시키기 위해서는

1) Firefox 브라우저가 깔려있어야 합니다.

2) geckodriver.exe가 현재 디렉토리에 있는지 확인해주세요.


In [5]:
def total_twitter_permonth(keyword,startyear,endyear):
    binary=FirefoxBinary('C:/Program Files/Mozilla Firefox/firefox.exe')
    browser=webdriver.Firefox(executable_path='geckodriver.exe',firefox_binary=binary)
    
    df = pd.DataFrame(data={'Date':[],'Frequency':[],'Heart':[],'Year':[],'Dong':[]})
    
    for loc in keyword:
        print(loc)
        for y in range(startyear,endyear+1):
            startdate = dt.date(year=y,month=1,day=1)
            untildate = dt.date(year=y,month=2,day=1)

            if y == 2018 :
                enddate = dt.date(year=y,month=9,day=1)
            else :
                enddate = dt.date(year=y+1,month=1,day=1)

            totalfreq=[]
            while not enddate == startdate:
                url='https://twitter.com/search?&q="'+str(loc)+'"%20since%3A'+str(startdate)+'%20until%3A'+str(untildate)+'&src=typd'
                browser.get(url)

                lastHeight = browser.execute_script("return document.body.scrollHeight")  # 스크롤 내리기 전 위치
                while True:
                    browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")  # 스크롤을 맨 아래까지 내림
                    time.sleep(4)  # 로딩하는 시간때문에 sleep을 걸어줍니다.

                    newHeight = browser.execute_script("return document.body.scrollHeight")  # 현재 스크롤 위치

                    if newHeight == lastHeight:  # 더이상 로딩될 트윗이 없다면
                        html = browser.page_source
                        soup = BeautifulSoup(html,'html.parser')

                        tweets = soup.find_all("p", {"class": "TweetTextSize"})
                        wordfreq = len(tweets)  # 페이지에 있는 트윗개수

                        heartfreq = 0
                        hearts = soup.find_all("span",text = re.compile("\d 마음"))
                        for hf in hearts:
                            heartfreq += int(hf.text[:-3].replace(',',''))

                        dailyfreq = {'Date':startdate,'Frequency':wordfreq,'Heart':heartfreq,'Year':startdate.year,'Dong':loc}
                        totalfreq.append(dailyfreq)

                        # 다음 작업을 위해 하루 늘려줌                  
                        startdate = untildate
                        untildate += relativedelta(months=1)
                        # 초기화
                        wordfreq = 0
                        dailyfreq = {}
                        break
                    lastHeight = newHeight

            df = pd.concat([df,pd.DataFrame(totalfreq)])
    return df

2009 ~ 2017

ss: 성수동 , mw: 망원동 , se: 서울대입구 or 샤로수길

sh: 신사동 , sc: 삼청동 , mg:명동

sd: 상도동 , mk: 목동 , sj:신정동 , sg:상계동

In [4]:
# loc_dict = {'성수동':'ss' , '망원동':'mw' , '샤로수길':'se', 
#             '신사동':'sh' , '삼청동':'sc' , '명동':'md',
#             '상도동':'sd' , '목동':'mk' , '명일동':'mi', '상계동':'sg'}

In [ ]:
# for loc in ['삼청동','명동','상도동','목동','명일동','상계동'] :
#     df = total_twitter_permonth(loc,2009,2017)
#     df.to_csv('df_'+loc_dict[loc]+'.csv',encoding='utf-8',index=False)

C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:41: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False



In [8]:
# df = total_twitter_permonth('가로수길',2009,2017)
# df.to_csv('df_garosu.csv',encoding='utf-8',index=False)

C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:41: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False



In [9]:
# df_ss = pd.read_csv('df_ss.csv')

In [15]:
# for n in range(8,19):
#     df_ss.loc[df_ss['Date'].str.contains('200'+str(n))|df_ss['Date'].str.contains('20'+str(n)),'Year'] = 2000+n

In [17]:
# df_ss.to_csv('df_ss.csv',encoding='utf-8-sig',index=False)

### 서울시 467개동

In [3]:
land = pd.read_csv('Report.csv',encoding='cp949')

In [4]:
dong_list = []
for d in land['법정동'].unique()[3:]:
    d = re.sub('\d가','',d)
    d = re.sub('\d동','동',d)
    dong_list.append(d)
dong_list = list(set(dong_list))
dong_list

['사당동',
 '의주로',
 '도렴동',
 '잠실동',
 '신촌동',
 '회현동',
 '명륜',
 '답십리동',
 '청파동',
 '무악동',
 '장충동',
 '양평동',
 '공릉동',
 '불광동',
 '필동',
 '신정동',
 '입정동',
 '서계동',
 '마천동',
 '논현동',
 '가양동',
 '장교동',
 '거여동',
 '삼성동',
 '마곡동',
 '수송동',
 '후암동',
 '홍은동',
 '대방동',
 '창전동',
 '도원동',
 '풍납동',
 '창천동',
 '창성동',
 '동자동',
 '신길동',
 '휘경동',
 '관수동',
 '항동',
 '신대방동',
 '중계동',
 '소공동',
 '송월동',
 '관철동',
 '신문로',
 '우이동',
 '남산동',
 '주자동',
 '염리동',
 '홍지동',
 '아현동',
 '통인동',
 '연지동',
 '구산동',
 '자곡동',
 '화동',
 '장지동',
 '대현동',
 '묵정동',
 '명일동',
 '수색동',
 '가락동',
 '수표동',
 '화곡동',
 '성수동',
 '장위동',
 '송현동',
 '청운동',
 '상계동',
 '금호동',
 '방화동',
 '남가좌동',
 '삼전동',
 '응봉동',
 '길동',
 '반포동',
 '시흥동',
 '남학동',
 '용산동',
 '부암동',
 '상도동',
 '독산동',
 '중곡동',
 '오쇠동',
 '군자동',
 '신계동',
 '자양동',
 '평동',
 '구수동',
 '누하동',
 '오장동',
 '하계동',
 '본동',
 '신당동',
 '성북동',
 '쌍문동',
 '다동',
 '창동',
 '사근동',
 '염창동',
 '오금동',
 '홍제동',
 '오류동',
 '순화동',
 '성내동',
 '청암동',
 '계동',
 '능동',
 '원효로',
 '봉익동',
 '산림동',
 '역촌동',
 '상왕십리동',
 '개포동',
 '내자동',
 '회기동',
 '효제동',
 '신내동',
 '상일동',
 '방산동',
 '공덕동',
 '종로',
 '봉천동',
 '연희동',
 '묘동',
 '응암동',


In [5]:
len(dong_list)

358

In [11]:
for n in range(1,71):
    df = total_twitter_permonth(dong_list[5*n:5*(n+1)],2009,2017)
    df.to_csv('twit_seoul/seoul_dong_'+str(n)+'.csv',encoding='utf-8-sig',index=False)

염곡동


C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:50: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False



내발산동
관수동
번동
교북동
이태원동
여의도동
보문동


In [13]:
df = total_twitter_permonth(dong_list[355:],2009,2017)
df.to_csv('twit_seoul/seoul_dong_72.csv',encoding='utf-8-sig',index=False)

이태원동


C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:50: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False



여의도동
보문동


In [ ]:
df = total_twitter_permonth(['익선동'],2009,2017)
df.to_csv('twit_seoul/seoul_dong_is.csv',encoding='utf-8-sig',index=False)

익선동


In [15]:
df = total_twitter_permonth(['연남동'],2009,2017)
df.to_csv('twit_seoul/seoul_dong_yn.csv',encoding='utf-8-sig',index=False)

연남동


In [6]:
df = total_twitter_permonth(['후암동'],2009,2017)
df.to_csv('twit_seoul/seoul_dong_ha.csv',encoding='utf-8-sig',index=False)

후암동


In [17]:
df = total_twitter_permonth(['성수동'],2009,2017)
df.to_csv('twit_seoul/seoul_dong_ss.csv',encoding='utf-8-sig',index=False)

성수동


In [10]:
df = total_twitter_permonth(['봉천동'],2009,2017)
df.to_csv('twit_seoul/seoul_dong_bc.csv',encoding='utf-8-sig',index=False)

봉천동


In [11]:
df = total_twitter_permonth(['망원동'],2009,2017)
df.to_csv('twit_seoul/seoul_dong_mw.csv',encoding='utf-8-sig',index=False)

망원동
